In [20]:
from decimal import Decimal, getcontext
import math

# Установка точности вычислений
getcontext().prec = 100

def read_data_kT(variant=1):
    data_tab = []
    with open('data/kT.txt') as file:
        for line in file:
            if line.strip():
                parts = line.split()
                T = Decimal(parts[1])
                k_val = Decimal(parts[2] if variant == 1 else parts[3])
                data_tab.append((T, k_val))
    return data_tab

def linear_interpolation(x, x_values, y_values):
    for i in range(len(x_values)-1):
        if x_values[i] <= x <= x_values[i+1]:
            return y_values[i] + (y_values[i+1] - y_values[i]) * (x - x_values[i]) / (x_values[i+1] - x_values[i])
    # Экстраполяция если x за границами
    if x < x_values[0]:
        return y_values[0]
    return y_values[-1]

def getkFunc(variantkT):
    data_kT = read_data_kT(variantkT)
    log_T = [d[0].ln() for d in data_kT]
    log_k = [d[1].ln() for d in data_kT]
    
    def k(T):
        T_dec = Decimal(str(T))
        x = T_dec.ln()
        interp_log_k = linear_interpolation(x, log_T, log_k)
        return interp_log_k.exp()
    
    return k

# Методы Рунге-Кутта
def RungeKutta4Step(duFunc, dFFunc, ri, ui, Fi, h):
    h = Decimal(str(h))
    ri = Decimal(str(ri))
    ui = Decimal(str(ui))
    Fi = Decimal(str(Fi))
    
    k1 = duFunc(ri, ui, Fi)
    q1 = dFFunc(ri, ui, Fi)
    
    h2 = h / Decimal('2')
    k2 = duFunc(ri + h2, ui + h2 * k1, Fi + h2 * q1)
    q2 = dFFunc(ri + h2, ui + h2 * k1, Fi + h2 * q1)
    
    k3 = duFunc(ri + h2, ui + h2 * k2, Fi + h2 * q2)
    q3 = dFFunc(ri + h2, ui + h2 * k2, Fi + h2 * q2)
    
    k4 = duFunc(ri + h, ui + h * k3, Fi + h * q3)
    q4 = dFFunc(ri + h, ui + h * k3, Fi + h * q3)
    
    yip1 = ui + (h/Decimal('6')) * (k1 + Decimal('2')*k2 + Decimal('2')*k3 + k4)
    zip1 = Fi + (h/Decimal('6')) * (q1 + Decimal('2')*q2 + Decimal('2')*q3 + q4)
    
    # if yip1 < 0:
    #     print(f"RK4step: yip1={float(yip1)}, k1={float(k1)} k2={float(k2)} k3={float(k3)} k4={float(k4)}, ri={float(ri)}, ui={float(ui)}, Fi={float(Fi)}")
    
    return yip1, zip1

def RungeKutta4(duFunc, dFFunc, r0, y0, z0, h, rTarget):
    r0 = Decimal(str(r0))
    y0 = Decimal(str(y0))
    z0 = Decimal(str(z0))
    h = Decimal(str(h))
    rTarget = Decimal(str(rTarget))
    
    rvals = [r0]
    yvals = [y0]
    Fvals = [z0]
    
    while rvals[-1] < rTarget:
        ri = rvals[-1] + h
        yi, Fi = RungeKutta4Step(duFunc, dFFunc, rvals[-1], yvals[-1], Fvals[-1], h)
        rvals.append(ri)
        yvals.append(yi)
        Fvals.append(Fi)
    
    return rvals, yvals, Fvals

# Константы и вспомогательные функции
c = Decimal('3') * Decimal('1e10')
R = Decimal('0.35')

def T_r(r):
    r = Decimal(str(r))
    Tw = Decimal('2000')
    To = Decimal('1e4')
    p = Decimal('4')
    return (Tw - To) * ((r / R) ** p) + To

def up_r(r):
    r = Decimal(str(r))
    const1 = Decimal('3.084E-4')
    const2 = Decimal('4.799E4')
    T = T_r(r)
    denominator = (const2 / T).exp() - Decimal('1')
    return const1 / denominator

k = getkFunc(variantkT=2)

r0 = Decimal('0')
def get_y0(khi):
    khi = Decimal(str(khi))
    return khi * up_r(r0)
F0 = Decimal('0')
rTarget = R
EPS = Decimal('1e-3')

def du(ri, ui, Fi):
    ri = Decimal(str(ri))
    ui = Decimal(str(ui))
    Fi = Decimal(str(Fi))
    k_val = k(T_r(ri))
    return -Decimal('3') * k_val * Fi / c

def dF(ri, ui, Fi):
    ri = Decimal(str(ri))
    ui = Decimal(str(ui))
    Fi = Decimal(str(Fi))
    k_val = k(T_r(ri))
    up = up_r(ri)
    
    if ri == Decimal('0') and Fi == Decimal('0'):
        return Decimal('0.5') * k_val * (up - ui)
    return -Fi / ri + c * k_val * (up - ui)

# Метод стрельбы и половинного деления
def solutionShooting(khi, RungeKuttaFunc, h_start):
    khi = Decimal(str(khi))
    h = Decimal(str(h_start))
    y0 = get_y0(khi)
    
    res_h = RungeKuttaFunc(du, dF, r0, y0, F0, h, rTarget)
    # yR_h = res_h[1][-1]
    
    # res_h2 = RungeKuttaFunc(du, dF, r0, y0, F0, h/Decimal('2'), rTarget)
    # yR_h2 = res_h2[1][-1]
    
    # while abs(yR_h - yR_h2) / yR_h2 >= EPS:
    #     print(f"khi = {khi}, h = {h}, yR_h = {yR_h}, yR_h2 = {yR_h2}, acc = {abs(yR_h - yR_h2) / yR_h2}")
    #     h /= Decimal('2')
    #     res_h, yR_h = res_h2, yR_h2
    #     res_h2 = RungeKuttaFunc(du, dF, r0, y0, F0, h/Decimal('2'), rTarget)
    #     yR_h2 = res_h2[1][-1]
    
    return (res_h[1][-1], res_h[2][-1]), h

def equationTarget(uR, FR):
    uR = Decimal(str(uR))
    FR = Decimal(str(FR))
    return FR - Decimal('0.39') * c * uR

def sign(val):
    if abs(val) < EPS:
        return 0
    elif val < 0:
        return -1
    return 1

def HalfDivMethod(RungeKuttaFunc, h_start):
    h = Decimal(str(h_start))
    khi_l, khi_r = Decimal('0'), Decimal('1')
    
    res_l, h = solutionShooting(khi_l, RungeKuttaFunc, h)
    res_r, h = solutionShooting(khi_r, RungeKuttaFunc, h)
    sign_l, sign_r = sign(equationTarget(*res_l)), sign(equationTarget(*res_r))

    while sign_l != sign_r:          # abs(khi_r - khi_l) / khi_l >= EPS:
        khi_mid = (khi_l + khi_r) / Decimal('2')
        res_mid, h = solutionShooting(khi_mid, RungeKuttaFunc, h)
        print(f"khim = {khi_mid}, yR = {res_mid[0]}, FR = {res_mid[1]}, h = {h}")
        sign_mid = sign(equationTarget(*res_mid))
        if sign_mid == 0:
            return khi_mid, h
        elif sign_mid == sign_l:
            khi_l = khi_mid
        else:
            khi_r = khi_mid
    
    return khi_mid, h

# Запуск расчета
h_st = Decimal('0.001')
khi_solution, h_solution = HalfDivMethod(RungeKutta4, h_st)
print(f"Решение: χ = {khi_solution}, h = {h_solution}")


khim = 0.5, yR = -165836350808810241260066964252.7364136849046833474930685001113880770162792927508403688487115205955377, FR = 2636785567944087016283188143948063134976.225180311464289912200705319731172547868432249896625965197357, h = 0.001
khim = 0.75, yR = -82917983561487278160670750247.09643613452516389216037790026816866420408739170263408670064769659302461, FR = 1318389733684005860169966926824622539987.738906538289937404566426773206519106747302633891993496463078, h = 0.001
khim = 0.875, yR = -41458799937825796610972643244.27644735933540416449403260034655895779799144117853094562661578459176795, FR = 659191816553965282113356318262902242493.4957696517027611507492874999441923861867378258896772620959325, h = 0.001
khim = 0.9375, yR = -20729208125995055836123589742.86645297174052430066085995038575410459494346591647937508959982859113965, FR = 329592857988944993085051013982042093746.3742012084091730238407178633130290259064554218885191449123622, h = 0.001
khim = 0.96875, yR = -1036441222007968

In [21]:
from decimal import Decimal
import plotly.express as px
import pandas as pd
import plotly.io as pio

# Установите nbformat если его нет: 
# !pip install nbformat>=4.2.0

# Альтернативный способ отображения графиков без nbformat
pio.renderers.default = "browser"  # Открывать графики в браузере

# Выбор метода и начальных условий
RungeKutta = RungeKutta4
y0 = get_y0(khi_solution)
print(f"khi = {khi_solution}, h = {h_solution}")

# Вычисление решений (все вычисления остаются в Decimal)
rvals, yvals, Fvals = RungeKutta(du, dF, r0, y0, F0, h_solution, rTarget)
upvals = [up_r(ri) for ri in rvals]

# Функция для безопасного преобразования Decimal в float для визуализации
def decimal_to_float(x):
    if isinstance(x, Decimal):
        return float(x)
    try:
        return float(x)
    except:
        return x

# Создание DataFrame
def create_decimal_df(rvals, yvals, label):
    return pd.DataFrame({
        "r": [decimal_to_float(r) for r in rvals],
        "f": [decimal_to_float(y) for y in yvals],
        "label": [label] * len(rvals)
    })

# Подготовка данных
df1 = create_decimal_df(rvals, yvals, "u(r)")
df2 = create_decimal_df(rvals, Fvals, "F(r)") 
df3 = create_decimal_df(rvals, upvals, "up(r)")

# Создание фигур
fig_u = px.line(
    pd.concat([df1, df3]), 
    x="r", 
    y="f", 
    color="label",
    title="Распределение плотности энергии излучения",
    labels={"r": "Радиус, см", "f": "Плотность энергии, Дж/см³"}
)

fig_F = px.line(
    df2,
    x="r",
    y="f",
    color="label",
    title="Распределение потока излучения",
    labels={"r": "Радиус, см", "f": "Поток излучения, Вт/см²"}
)

# Настройка внешнего вида
for fig in [fig_u, fig_F]:
    fig.update_layout(
        legend_title="",
        font=dict(size=12),
        plot_bgcolor='white',
        hovermode="x unified"
    )
    fig.update_traces(marker=dict(size=6))

# Сохранение и отображение графиков
fig_u.write_html("radiation_energy.html")
fig_F.write_html("radiation_flux.html")

print("Графики сохранены в HTML файлы:")
print("- radiation_energy.html")
print("- radiation_flux.html")

khi = 0.999998843182010143457982621240108022404296879951326874822596988954083564791346492269705017904311894, h = 0.001
Графики сохранены в HTML файлы:
- radiation_energy.html
- radiation_flux.html
